In [1]:
import sys
import os

# Get the path of the current notebook
notebook_dir = os.path.dirname(os.path.abspath("__file__"))

# Add the src directory to the system path
src_path = os.path.join(notebook_dir, '..', 'src')
sys.path.append(src_path)

In [2]:
import great_expectations as gx
import pandas as pd
import numpy as np
import os
from data import preprocess_data

In [3]:
df = pd.read_csv("../data/samples/sample.csv")
X, y = preprocess_data(df)
X.head()

/home/microwaved_frog/MLOps-for-music-popularity-prediction/.venv/lib/python3.10/site-packages/sklearn/preprocessing/_function_transformer.py:303: UserWarning: When set_output is configured to be 'pandas', func should return a pandas DataFrame to follow the set_output API  or feature_names_out should be defined.
  warnings.warn(warn_msg.format("pandas"))



,genres__a cappella,genres__abstract,genres__acoustic cover,genres__acoustic guitar cover,genres__adoracao,genres__adult standards,genres__afrobeat,genres__afrobeats,genres__afrofuturism,genres__afropop,...,dates__weekday_sin,dates__weekday_cos,dates__year,text__album_name_length,text__name_length,text__album_name_word_count,text__name_word_count,int__chart,int__mode,bool__explicit
0,False,False,False,False,False,False,False,False,False,False,...,0.974928,-0.222521,2018,15,52,2,10,0,1,False
1,False,False,False,False,False,False,False,False,False,False,...,0.781831,0.623490,1996,15,25,4,5,1,1,False
2,False,False,False,False,False,False,False,False,False,False,...,0.000000,1.000000,2014,13,26,3,6,0,0,False
3,False,False,False,False,False,False,False,False,False,False,...,0.974928,-0.222521,2001,16,21,3,4,0,1,False
4,False,False,False,False,False,False,False,False,False,False,...,-0.433884,-0.900969,2014,47,38,7,8,0,1,False


In [4]:
X

,genres__a cappella,genres__abstract,genres__acoustic cover,genres__acoustic guitar cover,genres__adoracao,genres__adult standards,genres__afrobeat,genres__afrobeats,genres__afrofuturism,genres__afropop,...,dates__weekday_sin,dates__weekday_cos,dates__year,text__album_name_length,text__name_length,text__album_name_word_count,text__name_word_count,int__chart,int__mode,bool__explicit
0,False,False,False,False,False,False,False,False,False,False,...,0.974928,-0.222521,2018,15,52,2,10,0,1,False
1,False,False,False,False,False,False,False,False,False,False,...,0.781831,0.623490,1996,15,25,4,5,1,1,False
2,False,False,False,False,False,False,False,False,False,False,...,0.000000,1.000000,2014,13,26,3,6,0,0,False
3,False,False,False,False,False,False,False,False,False,False,...,0.974928,-0.222521,2001,16,21,3,4,0,1,False
4,False,False,False,False,False,False,False,False,False,False,...,-0.433884,-0.900969,2014,47,38,7,8,0,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179935,False,False,False,False,False,False,False,False,False,False,...,0.000000,1.000000,2012,9,6,1,1,0,1,False
179936,False,False,False,False,False,False,False,False,False,False,...,-0.433884,-0.900969,2020,17,49,3,11,0,1,False
179937,False,False,False,False,False,False,False,False,False,False,...,-0.433884,-0.900969,2021,36,36,5,6,0,1,False
179938,False,False,False,False,False,False,False,False,False,False,...,-0.433884,-0.900969,1982,32,13,4,3,0,0,False


In [5]:
import great_expectations as gx
from great_expectations.data_context import FileDataContext
context = gx.get_context(context_root_dir="../services/gx")
context

FileDataContext loading fluent config


Loading 'datasources' ->
[{'assets': [...], 'name': 'sample', 'type': 'pandas'},
 {'assets': [...], 'name': 'sample_data', 'type': 'pandas'},
 {'assets': [...], 'name': 'features', 'type': 'pandas'}]
_PandasDataAsset.dict() - missing config_provider, skipping config substitution
_PandasDataAsset.dict() - missing config_provider, skipping config substitution
_PandasDataAsset.dict() - missing config_provider, skipping config substitution
Loaded 'sample' from fluent config
Loaded 'sample_data' from fluent config
Loaded 'features' from fluent config


{
  "anonymous_usage_statistics": {
    "enabled": true,
    "data_context_id": "658ab671-7357-4b71-8760-718815eecf13",
    "usage_statistics_url": "https://stats.greatexpectations.io/great_expectations/v1/usage_statistics",
    "explicit_id": true,
    "explicit_url": false
  },
  "checkpoint_store_name": "checkpoint_store",
  "config_variables_file_path": "uncommitted/config_variables.yml",
  "config_version": 3.0,
  "data_docs_sites": {
    "local_site": {
      "class_name": "SiteBuilder",
      "show_how_to_buttons": true,
      "store_backend": {
        "class_name": "TupleFilesystemStoreBackend",
        "base_directory": "uncommitted/data_docs/local_site/"
      },
      "site_index_builder": {
        "class_name": "DefaultSiteIndexBuilder"
      }
    }
  },
  "datasources": {},
  "evaluation_parameter_store_name": "evaluation_parameter_store",
  "expectations_store_name": "expectations_store",
  "fluent_datasources": {
    "sample": {
      "type": "pandas",
      "assets":

In [6]:
data_asset = context.get_datasource("features").get_asset("features_dataframe")
batch_request = data_asset.build_batch_request(dataframe=X)

context.add_or_update_expectation_suite("features_expectations")

context.list_expectation_suites()

[ExpectationSuiteIdentifier::expectation_suite,
 ExpectationSuiteIdentifier::features_expectations]

In [7]:
for i in X.columns:
    print(i, X[i].nunique())

genres__a cappella 2
genres__abstract 2
genres__acoustic cover 2
genres__acoustic guitar cover 2
genres__adoracao 2
genres__adult standards 2
genres__afrobeat 2
genres__afrobeats 2
genres__afrofuturism 2
genres__afropop 2
genres__agronejo 2
genres__alt z 2
genres__alternative dance 2
genres__alternative pop 2
genres__alternative rock 2
genres__ambient 2
genres__anime 2
genres__arrocha 2
genres__aussietronica 2
genres__australian dance 2
genres__australian indie 2
genres__avant-garde 2
genres__axe 2
genres__bachata 2
genres__background music 2
genres__background piano 2
genres__backing track 2
genres__bagpipe 2
genres__ballroom 2
genres__banda 2
genres__banjo 2
genres__baroque 2
genres__bassline 2
genres__bboy 2
genres__beatlesque 2
genres__beats 2
genres__bebop 2
genres__bgm 2
genres__big band 2
genres__big beat 2
genres__big room 2
genres__binaural 2
genres__black metal 2
genres__bluegrass 2
genres__blues 2
genres__blues rock 2
genres__bolero 2
genres__boom bap 2
genres__bossa nova 2


In [ ]:
validator = context.get_validator(
    batch_request=batch_request,
    expectation_suite_name="features_expectations"
)


for column in X.columns:
    # All columns should not have null values
    validator.expect_column_values_to_not_be_null(column)
    if column.startswith("genres") or column.startswith("multilabel") or column.startswith("categorical") or column.startswith("bool"):
        # Expect the column to be boolean (True or False)
        validator.expect_column_values_to_be_in_set(column, [True, False])

    elif column.endswith("_sin") or column.endswith("_cos"):
        validator.expect_column_values_to_be_between(column, min_value=-1, max_value=1)
        validator.expect_column_values_to_be_of_type(column, "float")

    elif column.endswith("year") or column.endswith("_length") or column.endswith("_count") or column.startswith("int"):
        # All columns should be positive
        validator.expect_column_min_to_be_between(column, min_value=0, max_value=None)
        # Column must be of type int
        validator.expect_column_values_to_be_of_type(column, "int")

    elif column.startswith("normal") or column.startswith("uniform"):
        # Column must be of type float
        validator.expect_column_values_to_be_of_type(column, "float")


    





In [9]:
validator.save_expectation_suite(discard_failed_expectations=False)

	1251 expectation(s) included in expectation_suite.


In [10]:
checkpoint = context.add_or_update_checkpoint(
    name = "features_test_checkpoint",
    validations = [
        {
            "batch_request": batch.batch_request,
            "expectation_suite_name": "features_expectations"
        } for batch in data_asset.get_batch_list_from_batch_request(batch_request)
    ]
)

In [11]:
# Print failed expectations
res = checkpoint.run()
res

	1251 expectation(s) included in expectation_suite.


Calculating Metrics:   0%|          | 0/6105 [00:00<?, ?it/s]

{
  "run_id": {
    "run_name": null,
    "run_time": "2024-07-12T10:53:20.617748+03:00"
  },
  "run_results": {
    "ValidationResultIdentifier::features_expectations/__none__/20240712T075320.617748Z/features-features_dataframe": {
      "validation_result": {
        "success": true,
        "results": [
          {
            "success": true,
            "expectation_config": {
              "expectation_type": "expect_column_values_to_not_be_null",
              "kwargs": {
                "column": "genres__a cappella",
                "batch_id": "features-features_dataframe"
              },
              "meta": {}
            },
            "result": {
              "element_count": 179940,
              "unexpected_count": 0,
              "unexpected_percent": 0.0,
              "partial_unexpected_list": [],
              "partial_unexpected_counts": [],
              "partial_unexpected_index_list": []
            },
            "meta": {},
            "exception_info": {